In [50]:
df = data.loc[:, ['title', 'url', 'text']]
df = df[df['text'].notnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1099
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1094 non-null   object
 1   url     1094 non-null   object
 2   text    1094 non-null   object
dtypes: object(3)
memory usage: 34.2+ KB


### csv로 저장

df.to_csv("./test.csv")

In [12]:
import konlpy
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import pickle
import csv

In [13]:
def title_preprocessing(text):
    if text.startswith('title'):
        return text[2]
    else:
        return text

def text_preprocessing(text):
    if text.startswith('text'):
        return text[3]
    else:
        return text

In [27]:
def data_read():
    # 데이터 읽어오기
    df = pd.read_csv('F:/eunjin/NLP/data/newslist_10.csv')

    # 전처리
    df['title'] = df['title'].apply(title_preprocessing)
    df['text'] = df['text'].apply(text_preprocessing)
    
    # 학습 데이터, 테스트 데이터 나누기
    title_list = df['title'].tolist()
    text_list = df['text'].tolist()
    from sklearn.model_selection import train_test_split

    # 학습 70%, 테스트 30%
    title_train, title_test, text_train, text_test = train_test_split(title_list, text_list, test_size=0.3, random_state=0)

    return text_test, text_train, title_train, title_test

In [22]:
# 형태소 분석
def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [23]:
# 학습
def learning_model(X_train, y_train, X_test, y_test):
    # 데이터 단어 사전 만들고 단어 빈도 벡터화하는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)

    # 문장별 나오는 단어 수 수치화, 벡터화
    logistic = LogisticRegression(C=10.0, penalty='12', random_state=0)

    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습
    pipe.fit(X_train, y_train)

    # 정확도
    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))

    # 모델 저장
    with open('pipe.dat', 'wb') as fp:
        pickle.dump(pipe, fp)
    
    print('save')

In [24]:
def using_model():
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)
    
    while True:
        text = input('텍스트를 작성해주세요: ')

        str = [text]
        accuracy = np.max(pipe.predict_proba(str) * 100)
        result = pipe.predict(str)[0]

        if result == '1':
            print('긍정')
        else :
            print('부정')
        
        print ('Accuracy: %.3f' % result)

In [30]:
def scrapping() :
    data_read()

# 학습
def learning() :
    title_train, title_test, text_train, text_test = data_read()
    learning_model(title_train, text_train, title_test, text_test)

# 사용
def using() :
    using_model()

In [32]:
learning()

NameError: name 'jpype' is not defined

In [34]:
using()

FileNotFoundError: [Errno 2] No such file or directory: 'pipe.dat'